In [ ]:
# Imports and Setup
import os, re, json, math, time, textwrap, pathlib, itertools
from pathlib import Path
from typing import Optional, Dict, Any, List
import requests
import pandas as pd

!mkdir -p data

DATA_ROOT = Path("/content/agny-SP/data").resolve()

In [ ]:
# Helpers
# --Log
def log(msg: str):
    print(f"[DL] {msg}")

# --Save
def save_bytes(content: bytes, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "wb") as f:
        f.write(content)
    log(f"Saved: {path} ({len(content):,} bytes)")

# --HTTP GET
def http_get(url: str, headers: Optional[Dict[str,str]] = None, params: Optional[Dict[str,Any]] = None, retries: int = 4, sleep: float = 1.0):
    last_err = None
    for i in range(retries):
        try:
            r = requests.get(url, headers=headers, params=params, timeout=60)
            r.raise_for_status()
            return r
        except Exception as e:
            last_err = e
            log(f"GET failed ({i+1}/{retries}) for {url}: {e}")
            time.sleep(sleep * (i+1))
    raise last_err

# --Pandas
def excel_to_csvs(xlsx_path: Path, out_dir: Path, preview_rows: int = 3):
    out_dir.mkdir(parents=True, exist_ok=True)
    xl = pd.ExcelFile(xlsx_path)
    shapes = {}
    for sheet in xl.sheet_names:
        df = xl.parse(sheet_name=sheet)
        safe_sheet = re.sub(r'[^A-Za-z0-9._-]+', '_', sheet).strip('_')
        csv_path = out_dir / f"{xlsx_path.stem}__{safe_sheet}.csv"
        df.to_csv(csv_path, index=False)
        shapes[sheet] = (df.shape[0], df.shape[1])
        log(f"  - wrote {csv_path.name}: {df.shape[0]} rows x {df.shape[1]} cols")
        if preview_rows > 0:
            with pd.option_context("display.width", 160, "display.max_columns", None):
                log(f"    Preview of '{sheet}':\n{df.head(preview_rows)}")
    return shapes


##Dataset Source Access


*   USGS ScienceBase
*   NYC Open Data (Socrata)
*   USDA



###USGS ScienceBase

In [ ]:
# ScienceBase
# --fetch item JSON, pull matching file
def sciencebase_fetch(item_id: str) -> Dict[str, Any]:
    url = f"https://www.sciencebase.gov/catalog/item/{item_id}?format=json"
    r = http_get(url)
    return r.json()

def sciencebase_download_excel(item_id: str, out_dir: Path, filename_match: Optional[str] = None):
    meta = sciencebase_fetch(item_id)
    files = meta.get("files") or []
    if not files:
        raise RuntimeError(f"No files listed for ScienceBase item {item_id}")

    downloaded = []
    for f in files:
        name = f.get("name") or ""
        url  = f.get("url")  or ""
        if not name or not url:
            continue
        if filename_match and filename_match.lower() not in name.lower():
            continue
        if not name.lower().endswith((".xlsx", ".xls")):
            continue

        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / name
        resp = http_get(url)
        save_bytes(resp.content, out_path)
        downloaded.append(out_path)
    if not downloaded:
        raise RuntimeError(f"No matching Excel files found for item {item_id} (filter={filename_match})")
    return downloaded


###Socrata (NYC Open Data)

In [ ]:
# NYC Open Data (Socrata)
# (for fewer rate limits) SOCRATA_APP_TOKEN = os.getenv("NYC_OPENDATA_APP_TOKEN", None)

def socrata_to_csv(resource_id: str, out_csv: Path, domain: str = "data.cityofnewyork.us", chunk: int = 50_000):
    """
    Paginate the SODA API and write a single CSV with all rows.
    """
    headers = {}
    if SOCRATA_APP_TOKEN:
        headers["X-App-Token"] = SOCRATA_APP_TOKEN

    out_csv.parent.mkdir(parents=True, exist_ok=True)
    first = True
    offset = 0
    total_rows = 0

    while True:
        params = {
            "$limit": chunk,
            "$offset": offset
        }
        url = f"https://{domain}/resource/{resource_id}.json"
        resp = http_get(url, headers=headers, params=params)
        rows = resp.json()
        if not rows:
            break
        df = pd.DataFrame(rows)
        mode = "w" if first else "a"
        header = first
        df.to_csv(out_csv, index=False, mode=mode, header=header)
        total_rows += len(df)
        log(f"{resource_id}: fetched {len(df):,} rows (offset {offset:,})")
        first = False
        offset += chunk


    if total_rows > 0:
        df_head = pd.read_csv(out_csv, nrows=5)
        log(f"Wrote {total_rows:,} rows to {out_csv.name}. Preview:\n{df_head}")
    else:
        log(f"No rows found for {resource_id}")


In [ ]:
MANIFEST = {
    # --- USGS ScienceBase items (Hydrologic Data Summaries) ---
    "USGS_LI_PineBarrens_2020": {
        "type": "sciencebase",
        "item_id": "62d9650bd34e2842e1edcf5a",
        "file_contains": "LIPineBarrens_2020_DataRelease.xlsx"  # hint for filtering
    },
    "USGS_LI_PineBarrens_2021": {
        "type": "sciencebase",
        "item_id": "63208bc8d34e71c6d67aa87e",
        "file_contains": "LIPineBarrens_2021_DataRelease.xlsx"
    },
    "USGS_LI_PineBarrens_2022": {
        "type": "sciencebase",
        "item_id": "64c7f088d34e70357a349e3b",
        "file_contains": "LIPineBarrens_2022_DataRelease.xlsx"
    },

    # --- Bronx River PFAS/pesticides/pharmaceuticals (ScienceBase) ---
    "USGS_BronxRiver_2019_PFAS_Pest_Pharma": {
        "type": "sciencebase",
        "item_id": "62d96512d34e2842e1edcf5e",
        "file_contains": "PFAS_pesticides_pharmaceuticals_BronxRiver2019_NYC_NY_all_data.xlsx"
    },

    # --- Western NY & Mohawk River Basin Groundwater 2021 (ScienceBase) ---
    "USGS_WNY_Mohawk_2021": {
        "type": "sciencebase",
        "item_id": "65203b86d34e44db0e2e4406",
        "file_contains": ".xlsx"  # catch the main Excel
    },

    # --- Long Island shallow groundwater 2016-2018 (ScienceBase) ---
    "USGS_LI_ShallowGW_2016_2018": {
        "type": "sciencebase",
        "item_id": "5e83a6b4e4b01d50927b62a9",
        "file_contains": ".xlsx"
    },

    # --- NYC Open Data (Socrata) ---
    "NYC_DEP_Distribution_WQ": {
        "type": "socrata",
        "resource_id": "bkwf-xfky",
        "outfile": "Drinking_Water_Quality_Distribution_Monitoring_Data.csv"
    },
    "NYC_DEP_Watershed_WQ": {
        "type": "socrata",
        "resource_id": "y43c-5n92",
        "outfile": "Watershed_Water_Quality_Data.csv"
    },
    "NYC_DEP_Crypto_Giardia": {
        "type": "socrata",
        "resource_id": "x2s6-6d2j",
        "outfile": "DEP_Cryptosporidium_And_Giardia_Data_Set.csv"
    },

    # --- USDA/NASS CSV (direct link downloads you provided) ---
    "USDA_NASS_Regional_Crop_Yield": {
        "type": "direct_csv",
        # try the 'printable' link first; if it fails, try the results GUID link
        "urls": [
            "https://quickstats.nass.usda.gov/data/printable/F749339A-E2BC-3A10-BFCC-91B6686691E0",
            "https://quickstats.nass.usda.gov/results/3D668DBB-3D36-3013-8052-B1F3C7580171"
        ],
        "outfile": "USDA_Regional_Crop_Yield_Data.csv"
    },
}


In [ ]:
def run_manifest(manifest: Dict[str, Dict[str, Any]], root: Path = DATA_ROOT):
    summary = {}

    for key, cfg in manifest.items():
        log(f"=== {key} ===")
        out_dir = root / key
        kind = cfg["type"]

        if kind == "sciencebase":
            item_id = cfg["item_id"]
            contains = cfg.get("file_contains")
            xls_paths = sciencebase_download_excel(item_id, out_dir, filename_match=contains)
            sheet_shapes_all = {}
            for p in xls_paths:
                csv_dir = out_dir / "csv"
                shapes = excel_to_csvs(p, csv_dir)
                sheet_shapes_all[p.name] = shapes
            summary[key] = {
                "excel_files": [str(p) for p in xls_paths],
                "sheet_shapes": sheet_shapes_all
            }

        elif kind == "socrata":
            rid = cfg["resource_id"]
            outfile = cfg["outfile"]
            out_csv = out_dir / outfile
            socrata_to_csv(rid, out_csv)
            summary[key] = {"csv": str(out_csv)}

        elif kind == "direct_csv":
            urls = cfg["urls"]
            outfile = cfg["outfile"]
            out_csv = out_dir / outfile
            out_dir.mkdir(parents=True, exist_ok=True)
            last_err = None
            for u in urls:
                try:
                    r = http_get(u)
                    # Some QuickStats "printable" links return text/HTML – try to coerce to CSV if it is CSV.
                    # If it's HTML, pandas can often still read the table; but we prefer raw content save first.
                    content = r.content
                    # Quick sanity: if comma-heavy and linebreaks exist, we’ll just save as CSV.
                    save_bytes(content, out_csv)
                    # optional: try reading few rows to confirm
                    try:
                        df = pd.read_csv(out_csv, nrows=5)
                        log(f"Preview of {outfile}:\n{df}")
                    except Exception as e:
                        log(f"Note: Could not parse preview as CSV yet ({e}). File still saved.")
                    last_err = None
                    break
                except Exception as e:
                    last_err = e
                    log(f"Direct CSV download failed for {u}: {e}")
            if last_err:
                raise last_err
            summary[key] = {"csv": str(out_csv)}
        else:
            log(f"Unknown type: {kind}")

    return summary

summary = run_manifest(MANIFEST)
log("\n=== DONE ===")
print(json.dumps(summary, indent=2)[:2000])  # trim preview
